# Объединение энергопотребления с погодными данными

Этот тестовый ноутбук показывает, как объединить почасовые данные по энергопотреблению
с почасовыми погодными данными `dfw_hourly`.

- Источник энергопотребления: `DataManager.get_clean()` (таргет `Usage_kWh`)
- Источник погоды: файл `data/raw/weather_dfw_hourly.parquet`, сохраненный из `weather_data.ipynb`

Результат: объединенный датафрейм с энергопотреблением и погодными признаками по общему
почасовому индексу времени.



In [1]:
# Настройка путей и импортов
import sys
from pathlib import Path

# Находим корень проекта (для ноутбука в notebooks/experiments)
project_root = Path().resolve().parent.parent
sys.path.append(str(project_root))

import pandas as pd
import numpy as np

from src.data.data_manager import DataManager

# Инициализация DataManager и загрузка энергопотребления
dm = DataManager()

df_energy = dm.get_clean().copy()
# Убедимся, что индекс — это DateTime и отсортирован
if not isinstance(df_energy.index, pd.DatetimeIndex):
    df_energy.index = pd.to_datetime(df_energy.index)

df_energy = df_energy.sort_index()
df_energy.index.name = "DateTime"

print("Форма df_energy:", df_energy.shape)
df_energy.head()


 Clean данные загружены из кеша: 35064 строк
Форма df_energy: (35064, 1)


,Usage_kWh
DateTime,
2017-01-01 00:00:00,570.685479
2017-01-01 01:00:00,604.642705
2017-01-01 02:00:00,518.732113
2017-01-01 03:00:00,608.188829
2017-01-01 04:00:00,714.140572


In [3]:
# Загрузка погодных данных dfw_hourly
from pathlib import Path

weather_parquet_path = Path("..") / ".." / "data" / "raw" / "weather_dfw_hourly.parquet"
print("Путь к погодным данным:", weather_parquet_path)

df_weather = pd.read_parquet(weather_parquet_path)

# Убедимся, что индекс — DatetimeIndex и называется так же, как в энергоданных
if not isinstance(df_weather.index, pd.DatetimeIndex):
    df_weather.index = pd.to_datetime(df_weather.index)

# Переименуем индекс для согласованности
df_weather = df_weather.sort_index().copy()
df_weather.index.name = "DateTime"

print("Форма df_weather:", df_weather.shape)
df_weather.head()


Путь к погодным данным: ../../data/raw/weather_dfw_hourly.parquet
Форма df_weather: (35064, 3)


,wind_speed,T,f
DateTime,,,
2017-01-01 00:00:00,2.0,2.6,88.0
2017-01-01 01:00:00,2.0,2.6,88.0
2017-01-01 02:00:00,2.0,2.6,88.0
2017-01-01 03:00:00,4.0,2.6,82.0
2017-01-01 04:00:00,4.0,2.6,82.0


In [5]:
# Объединение энергопотребления и погодных данных по времени

# Внутреннее объединение по общему часовому индексу

df_merged = df_energy.join(df_weather, how="inner")

print("Форма объединенного df_merged:", df_merged.shape)
df_merged.head()


Форма объединенного df_merged: (35064, 4)


,Usage_kWh,wind_speed,T,f
DateTime,,,,
2017-01-01 00:00:00,570.685479,2.0,2.6,88.0
2017-01-01 01:00:00,604.642705,2.0,2.6,88.0
2017-01-01 02:00:00,518.732113,2.0,2.6,88.0
2017-01-01 03:00:00,608.188829,4.0,2.6,82.0
2017-01-01 04:00:00,714.140572,4.0,2.6,82.0


In [7]:
# Быстрая проверка пропусков по основным колонкам

print("Пропуски по Usage_kWh:")
print(df_merged["Usage_kWh"].isnull().sum())

print("\nПропуски по погодным признакам:")
for col in ["wind_speed", "T", "f"]:
    if col in df_merged.columns:
        print(col, ":", df_merged[col].isnull().sum())


Пропуски по Usage_kWh:
0

Пропуски по погодным признакам:
wind_speed : 0
T : 0
f : 0


In [9]:
# (Опционально) Сохранение объединенного датафрейма для дальнейших экспериментов
from pathlib import Path

merged_path = Path("..") / ".." / "data" / "views" / "energy_weather_merged.parquet"
merged_path.parent.mkdir(parents=True, exist_ok=True)

df_merged.to_parquet(merged_path)
print("Объединенный датасет сохранен в:", merged_path)


Объединенный датасет сохранен в: ../../data/views/energy_weather_merged.parquet
